In [13]:
using Pkg
Pkg.activate("..")
Pkg.add("Flux")

  Activating project at `~/school/t`
   Resolving package versions...
  No Changes to `~/school/t/Project.toml`
  No Changes to `~/school/t/Manifest.toml`


In [14]:
using JLD2
X_train = load("./data_rnn/imdb_dataset_prepared.jld2", "X_train")
y_train = load("./data_rnn/imdb_dataset_prepared.jld2", "y_train")
X_test = load("./data_rnn/imdb_dataset_prepared.jld2", "X_test")
y_test = load("./data_rnn/imdb_dataset_prepared.jld2", "y_test")
embeddings = load("./data_rnn/imdb_dataset_prepared.jld2", "embeddings")
vocab = load("./data_rnn/imdb_dataset_prepared.jld2", "vocab")
nothing

embedding_dim = size(embeddings,1);

In [15]:
using Flux, Random
Random.seed!(0)

model = Chain(
    Flux.Embedding(length(vocab), embedding_dim),
    Flux.RNN(embedding_dim => 16, relu, return_state = true),
    x -> x[end],
    Flux.flatten,
    Dense(16, 1, σ)
)


Chain(
  Embedding(12849 => 50),               # 642_450 parameters
  RNN(50 => 16, relu),                  # 1_072 parameters
  var"#19#20"(),
  Flux.flatten,
  Dense(16 => 1, σ),                    # 17 parameters
)                   # Total: 6 arrays, 643_539 parameters, 2.455 MiB.

In [16]:
# add Glove embeddings to Embedding layer
model.layers[1].weight .= embeddings;

In [17]:
using Printf, Statistics

dataset = Flux.DataLoader((X_train, y_train), batchsize=128, shuffle=true)

loss(m, x, y) = Flux.Losses.binarycrossentropy(m(x), y)
accuracy(m, x, y) = mean((m(x) .> 0.5) .== (y .> 0.5))

opt = Optimisers.setup(RMSProp(), model)

epochs = 12
for epoch in 1:epochs
    total_loss = 0.0
    total_acc = 0.0
    num_samples = 0

    t = @time begin
        for (x, y) in dataset
            Flux.reset!(model)
            grads = Flux.gradient(model) do m
                loss(m, x, y)
            end
            Optimisers.update!(opt, model, grads[1])
            total_loss += loss(model, x, y)
            total_acc += accuracy(model, x, y)
            num_samples += 1
        end

        train_loss = total_loss / num_samples
        train_acc = total_acc / num_samples

        test_acc = accuracy(model, X_test, y_test)
        test_loss = loss(model, X_test, y_test)
    end

    println(@sprintf("Epoch: %d (%.2fs) \tTrain: (l: %.2f, a: %.2f) \tTest: (l: %.2f, a: %.2f)", 
        epoch, t, train_loss, train_acc, test_loss, test_acc))
end

  5.013149 seconds (15.54 M allocations: 13.065 GiB, 15.40% gc time, 9.99% compilation time)
Epoch: 1 (0.69s) 	Train: (l: 0.69, a: 0.52) 	Test: (l: 0.69, a: 0.50)
  4.580736 seconds (13.18 M allocations: 12.952 GiB, 17.46% gc time)
Epoch: 2 (0.69s) 	Train: (l: 0.68, a: 0.54) 	Test: (l: 0.69, a: 0.51)
  4.576097 seconds (13.18 M allocations: 12.952 GiB, 17.51% gc time)
Epoch: 3 (0.53s) 	Train: (l: 0.57, a: 0.71) 	Test: (l: 0.53, a: 0.75)
  4.626041 seconds (13.18 M allocations: 12.952 GiB, 17.43% gc time)
Epoch: 4 (0.47s) 	Train: (l: 0.46, a: 0.80) 	Test: (l: 0.47, a: 0.79)
  4.542310 seconds (13.18 M allocations: 12.952 GiB, 17.06% gc time)
Epoch: 5 (0.42s) 	Train: (l: 0.41, a: 0.83) 	Test: (l: 0.42, a: 0.82)
  4.486685 seconds (13.18 M allocations: 12.952 GiB, 16.68% gc time)
Epoch: 6 (0.39s) 	Train: (l: 0.36, a: 0.86) 	Test: (l: 0.39, a: 0.84)
  4.608533 seconds (13.18 M allocations: 12.952 GiB, 17.79% gc time)
Epoch: 7 (0.38s) 	Train: (l: 0.32, a: 0.88) 	Test: (l: 0.38, a: 0.85)
  4